In [1]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2022-01-21
# GitHub: https://github.com/jaaack-wang 

## Quick start

With wrapped up functions that we will gradually learn throughout this tutorial, preprocessing the text data into one that is ready for model training can be as simple as following. Does it really work? Let's explore!

In [2]:
from utils import load_dataset, gather_text
from paddle_utils import * 

train_set = load_dataset('train.tsv')

text = gather_text(train_set)
V = TextVectorizer()
V.build_vocab(text)

trans_fn = get_trans_fn(V, include_seq_len=False)
batchify_fn = get_batchify_fn(include_seq_len=False)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w9/d_nplhzj4qx35xxlgljgdtjh0000gn/T/jieba.cache
Loading model cost 0.798 seconds.
Prefix dict has been built successfully.


Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


In [3]:
for example in train_loader:
    print(example)
    break

[Tensor(shape=[64, 254], dtype=int64, place=CPUPlace, stop_gradient=True,
       [[1461 , 2    , 8    , ..., 0    , 0    , 0    ],
        [22   , 76   , 2    , ..., 0    , 0    , 0    ],
        [426  , 28   , 3    , ..., 0    , 0    , 0    ],
        ...,
        [8    , 25   , 15072, ..., 0    , 0    , 0    ],
        [315  , 3    , 215  , ..., 0    , 0    , 0    ],
        [15   , 86   , 255  , ..., 0    , 0    , 0    ]]), Tensor(shape=[64], dtype=int64, place=CPUPlace, stop_gradient=True,
       [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1])]


## Overview

In this tutorial, we will use functions from `paddle` to help us preprocess and numericalize datasets. As these functions are native to `paddle`, so there is an advantage in training models constructed by using `paddle`, especially when the datasets are large. You will also need [`paddlenlp`](https://github.com/PaddlePaddle/PaddleNLP), a nlp package designed by the `paddle` team, to get everything going. To download it, simply run in command `pip3 install --upgrade paddlenlp`.

If you need more intuition about the ins and outs of this process, please refer to `2 - preprocess_data.ipynb` in the same folder.

Below are the structure of this tutoiral:

- [Load dataset](#1)
- [Create vocab_to_idx mapping dictionary](#2)
- [Text encoder](#3)
- [Example converter](#4)
    - [Coverting multiple examples](#4-1)
- [Creating dataloader](#5)
    - [Transform the dataset into  Dataset class using MapDataset](#5-1)
    - [A data sampler](#5-2)
    - [Building a batchify method](#5-3)
    - [Now the dataloader](#5-4)
- [A quick test](#6)
- [Wrapped up functions](#7)
    - [TextVectorizer](#7-1)
    - [Example converter](#7-2)
    - [Get trans_fn](#7-3)
    - [Get batchify_fn](#7-4)
    - [Create dataloader](#7-5)
- [More thorough tests](#8)
    - [Initializations](#8-1)
    - [Test One: CNN](#8-2)
    - [Test Two: RNN](#8-3)

<a name="1"></a>
## Load dataset

As usual, let's first use the `load_dataset` function compiled in the last two tutorials to load the datasets.

In [4]:
from utils import load_dataset

train_set = load_dataset('train.tsv')

# check. should be 4000 (recall `1 - get_data.ipynb`)
len(train_set)

4000

<a name="2"></a>

## Create `vocab_to_idx` mapping dictionary

The purpose of creating a `vocab_to_idx` mapping dictionary is for later encoding or numeralizing text data for model training. In the `2.1 - wrapped_up_data_preprocessor` tutorial, we have learnt how to use `TextVectorizer` to conveniently do this job. 

In this tutorial, we will use `paddlenlp.data.Vocab` to do a simlar job, but we will still use `jieba` to tokenize Chinese. Although `paddlenlp.data.JiebaTokenizer` can also be used, we will need a vocabulary to use it together with `Vocab` to make it work, which is not a general method to deal with other languages as well. Since we will need to install `jieba` anyway, let's go with a simpler and more generally applicable way.

Unfortunately, unlike the `TextVectorize` we have built, `paddlenlp.data.Vocab` do not have a text_encoder function for us to use directly, but we can built it ourself.

In [5]:
from paddlenlp.data import Vocab
import jieba

In [6]:
# First, we need a tokenize func
tokenize = jieba.lcut

# Then we need a list of tokenized texts
from utils import gather_text
text = gather_text(train_set) # ---> gather text from the train_set
tokens = list(map(tokenize, text)) # ---> a list of tokenized texts ([[w1, w2...], [w1, w2...]...])

# build the vocabulary which will give us the mapping dictionaries for encoding
V = Vocab.build_vocab(tokens, unk_token='[UNK]', pad_token='[PAD]')

In [7]:
# Let's see what we have 
# The most useful will be: V.token_to_idx and V.idx_to_token
[d for d in dir(V) if not d.startswith("_")]

['bos_token',
 'build_vocab',
 'eos_token',
 'from_dict',
 'from_json',
 'idx_to_token',
 'load_vocabulary',
 'pad_token',
 'to_indices',
 'to_json',
 'to_tokens',
 'token_to_idx',
 'unk_token']

In [8]:
# check

tmp = "{:20}{}"
print("The first 10 examples from the token_to_idx dictionary\n")

for item in list(V.token_to_idx.items())[:10]:
    print(tmp.format(*item))
    
    
print("\n\nThe first 10 random examples from the idx_to_token dictionary\n")

for idx, tk in list(V.idx_to_token.items())[:10]:
    print(tmp.format(str(idx), tk))

The first 10 examples from the token_to_idx dictionary

[PAD]               0
[UNK]               1
，                   2
的                   3
。                   4
了                   5
                    6
是                   7
我                   8
,                   9


The first 10 random examples from the idx_to_token dictionary

0                   [PAD]
1                   [UNK]
2                   ，
3                   的
4                   。
5                   了
6                    
7                   是
8                   我
9                   ,


In [9]:
# for unseen token_to_idx

print("Index for \033[1mThis_Word_Does_Not_Exist\033[0m:", V.token_to_idx["This_Word_Does_Not_Exist"])

Index for This_Word_Does_Not_Exist: 1


<a name="3"></a>
## Text encoder

If you are interested, you can also build a `text_decoder` as we did before. 

In [10]:
def text_encoder(text, 
                 tokenize=tokenize, 
                 token_to_idx=V.token_to_idx):
    
    tokens = tokenize(text)
    out = []
    for tk in tokens:
        out.append(token_to_idx[tk])
        
    return out

In [11]:
# check

print("Original text:", text[0])
print("Encoded text:", text_encoder(text[0]))

Original text: 选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
Encoded text: [189, 9545, 1205, 3, 429, 37, 99, 2, 17, 18740, 16391, 348, 917, 1944, 2, 606, 2482, 27, 21892, 27, 2185, 27, 1209, 27, 16748, 5476, 4, 13, 291, 76, 2, 44, 21, 260, 1071, 4, 6, 2870, 14, 284, 3, 4818, 2, 1102, 427, 2, 96, 399, 497, 7, 59, 4, 6, 533, 3, 108, 7, 2920, 3, 2, 21, 260, 472, 4, 6, 43, 320, 2, 76]


<a name="4"></a>
## Example converter 

We will see why this is needed shortly. The purpose of an example converter is to transform a given example into the data we need for training models. Later, we will use `map` function to map the `example_converter` to an entire dataset.

If you see the previous tutorials, we will know that since the `RNN` models takes as an input the text seq length, we also need to take that into consideration. More generally, as different tasks or different models have different needs, you need to tailor the `example_converter` based on your specific needs.

In [12]:
def example_converter(example, text_encoder, include_seq_len):
    
    text, label = example
    encoded = text_encoder(text)
    if include_seq_len:
        text_len = len(encoded)
        return encoded, text_len, label
    return encoded, label

In [13]:
# check

encoded, label = example_converter(train_set[0], text_encoder, False)
t, l = train_set[0]

print("Original text:", t)
print("Encoded text:", encoded)

# nothing changes for label

Original text: 选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
Encoded text: [189, 9545, 1205, 3, 429, 37, 99, 2, 17, 18740, 16391, 348, 917, 1944, 2, 606, 2482, 27, 21892, 27, 2185, 27, 1209, 27, 16748, 5476, 4, 13, 291, 76, 2, 44, 21, 260, 1071, 4, 6, 2870, 14, 284, 3, 4818, 2, 1102, 427, 2, 96, 399, 497, 7, 59, 4, 6, 533, 3, 108, 7, 2920, 3, 2, 21, 260, 472, 4, 6, 43, 320, 2, 76]


<a name="4-1"></a>
### Coverting multiple examples

To convert multiple examples, we can either loop through the examples one by one or use the `map` function. The `map` function takes a function as the first input and then one or several iterable (e.g., list or tuple) corresponding to the parameters of that function. In our `example_converter` we have three parameters, that means we will need to do something like the following:

```python
>>> examples = 'a list of examples'
>>> n = len(examples)
>>> E = text_encoder
>>> B = include_seq_len
>>> converted = map(example_converter, examples, [E] * n, [B] * n)
```

That looks bad, right? If you do not multiply the later two parameters, making them iterable, we will see something like the following:

In [14]:
examples = train_set[:2]
converted = map(example_converter, examples, text_encoder, False)

TypeError: 'function' object is not iterable

**There are three ways to get around this:**

- First, assign a default value to the latter two parameters (text_encoder & include_seq_len) so that you only need to pass the "example" paramters when using `map`. If we want to change the default values, we need to do it every time by changing the `example_converter` function directly.

- Second, use the handy built-in function `partial` from `functools` or `lambda` to assign default values for a function's parameters, which will create another encapsulated function for you. 

- Third, in our case, we can simply make the `example_converter` return the text seq length regardless, but that means, you need to construct all your models taking this into account (some models that do not need seq length info will have the related parameters but never use them). 


Below, we will show the second way because it is what is commonly adopted by the deep learning community when doing natural language processing.

#### Use `lambda`

In [15]:
trans_fn = lambda example: example_converter(example, text_encoder, False)
converted = map(trans_fn, examples)

for idx, conv in enumerate(converted):
    print(f"Example #{idx+1}....")
    e, l = conv
    print("Encoded text:", e)
    print("Label:", l)

Example #1....
Encoded text: [189, 9545, 1205, 3, 429, 37, 99, 2, 17, 18740, 16391, 348, 917, 1944, 2, 606, 2482, 27, 21892, 27, 2185, 27, 1209, 27, 16748, 5476, 4, 13, 291, 76, 2, 44, 21, 260, 1071, 4, 6, 2870, 14, 284, 3, 4818, 2, 1102, 427, 2, 96, 399, 497, 7, 59, 4, 6, 533, 3, 108, 7, 2920, 3, 2, 21, 260, 472, 4, 6, 43, 320, 2, 76]
Label: 1
Example #2....
Encoded text: [3289, 963, 416, 3, 175, 310, 2250, 2, 306, 241, 1900, 644, 5, 2, 682, 59, 2610, 3999, 2, 10320, 2610, 2423, 99, 2, 611, 12, 10, 2905, 2, 345, 12, 384, 26]
Label: 1


#### Use `partial`

In [16]:
from functools import partial 

# names for paramters you want to set default must be given
trans_fn = partial(example_converter, 
                   text_encoder=text_encoder, 
                   include_seq_len=False)

converted = map(trans_fn, examples)

for idx, conv in enumerate(converted):
    print(f"Example #{idx+1}....")
    e, l = conv
    print("Encoded text:", e)
    print("Label:", l)

Example #1....
Encoded text: [189, 9545, 1205, 3, 429, 37, 99, 2, 17, 18740, 16391, 348, 917, 1944, 2, 606, 2482, 27, 21892, 27, 2185, 27, 1209, 27, 16748, 5476, 4, 13, 291, 76, 2, 44, 21, 260, 1071, 4, 6, 2870, 14, 284, 3, 4818, 2, 1102, 427, 2, 96, 399, 497, 7, 59, 4, 6, 533, 3, 108, 7, 2920, 3, 2, 21, 260, 472, 4, 6, 43, 320, 2, 76]
Label: 1
Example #2....
Encoded text: [3289, 963, 416, 3, 175, 310, 2250, 2, 306, 241, 1900, 644, 5, 2, 682, 59, 2610, 3999, 2, 10320, 2610, 2423, 99, 2, 611, 12, 10, 2905, 2, 345, 12, 384, 26]
Label: 1


<a name="5"></a>
## Creating dataloader

Now comes with the most important points! **I figure that detailed explanations may not help you to understand what will be shown below, because you may need to practice again and again, and compare with what we have done previously to build a solid intuition.** Let's simply take a dataloder as a black box. All you need to know is what needs to go in and what will come out. Here are some of the points you need to know:

- A dataloader is something iterable and will work more efficiently with the models constructed by a deep learning framework, especially when trained on GPUs because they can load data asynchronously.


- For a dataloader, you need to pass the dataset you want to train on, the `example_converter` (or the encapsulated `trans_fn`) , a data `sampler` (i.e., how to build batches), as well as a `batchify` method to preprocess a given batch (or sample).


- For the dataset, its type needs to be what is called `Dataset` (map-style dataset) or `IterableDataset` (iterable-style dataset) in order to make everything work. 


Enough words. Let's just see what this can be done.

<a name="5-1"></a>
### Transform the dataset into  `Dataset` class using `MapDataset`

Some properties of the `Dataset`:

- It is iterable both by a for loop and by a slicing index (just like a list!)
- It has a `Dataset.map` method that will transform (namely numericalize) the entire dataset given a `trans_fn` (with only one paratemter allowed)
- The transformed `Dataset` will be iterable by a for loop but can not be indexed anymore.

In [17]:
import paddle
from paddlenlp.datasets import MapDataset

train = MapDataset(train_set)

print("Type of train", type(train))
print("Is train's type a Dataset?", isinstance(train, paddle.io.Dataset))

Type of train <class 'paddlenlp.datasets.dataset.MapDataset'>
Is train's type a Dataset? True


In [18]:
from copy import deepcopy # we need "deepcopy" to protect the orignal data

train_copy = deepcopy(train)
train_copy.map(trans_fn)

for idx, emp in enumerate(train_copy):
    print(f"Number {idx+1} example: ", emp)
    if idx == 2:
        break
        
# versus (remeber, after transformation, we can't use slicing index!)
print("\nThe first three examples...\n", train[:3])

Number 1 example:  ([189, 9545, 1205, 3, 429, 37, 99, 2, 17, 18740, 16391, 348, 917, 1944, 2, 606, 2482, 27, 21892, 27, 2185, 27, 1209, 27, 16748, 5476, 4, 13, 291, 76, 2, 44, 21, 260, 1071, 4, 6, 2870, 14, 284, 3, 4818, 2, 1102, 427, 2, 96, 399, 497, 7, 59, 4, 6, 533, 3, 108, 7, 2920, 3, 2, 21, 260, 472, 4, 6, 43, 320, 2, 76], 1)
Number 2 example:  ([3289, 963, 416, 3, 175, 310, 2250, 2, 306, 241, 1900, 644, 5, 2, 682, 59, 2610, 3999, 2, 10320, 2610, 2423, 99, 2, 611, 12, 10, 2905, 2, 345, 12, 384, 26], 1)
Number 3 example:  ([22, 505, 4, 164, 3, 16, 76, 4, 4, 4, 4, 4, 4, 4, 4, 4], 0)

The first three examples...
 [('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 1), ('15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 1), ('房间太小。其他的都一般。。。。。。。。。', 0)]


<a name="5-2"></a>
### A data sampler

There are two types of data sampler in `paddle`. One is `paddle.io.DistributedBatchSampler` for distributed multiple GPU training. And the other is `paddle.io.BatchSampler` for the normal use. We will use the later here.

Please note that the batched dataset will be only iterable in a for loop, but we can use the `list` function nested within a `len` function to count how many batches there are. For every batch inside this iterable output are a list of indices that index the dataset. For example, an index "1" would mean "dataset\[1\]". The indices will later be used by the dataloader to retrieve examples from the transformed/numerilized dataset. 

In [19]:
from paddle.io import BatchSampler

# do not forget to map the "trans_fn" to the "train" dataset. with the "map" func applied,
# you can no longer run this cell again, as the "train" has been transformed and no longer has "map" (see above)
bacth_sampler = BatchSampler(dataset=train.map(trans_fn), shuffle=True, batch_size=64)

# check: 4000 == 63 * 64 + 32
print("Number of batches:", len(list(bacth_sampler)))
print("Number of items in the last batch:",  len(list(bacth_sampler)[-1]))
print("Everything all right?", 4000 == 64 * 62 + 32)

Number of batches: 63
Number of items in the last batch: 32
Everything all right? True


In [20]:
first_10 = list(bacth_sampler)[0][:10]

print("This is the first 10 examples from the first batch:", first_10)
print("The indices are connecting with the transformed/numericalized dataset.")

This is the first 10 examples from the first batch: [547, 2074, 2973, 1834, 795, 2871, 1477, 522, 2022, 2087]
The indices are connecting with the transformed/numericalized dataset.


<a name="5-3"></a>
### Building a `batchify` method 

The purpose of the `batchify` method is to provide a set of methods to further preprocess the bacthed dataset in a way that make possible model training. More concretely, the following are three things we need to consider:

- for the text ids (numericalized text) within a batch, we need to make sure that they are of same length/dimension (aligned with the max length in a batch). We can use the `paddlenlp.data.Pad` function to do that.
- for the label values or the likes within a batch, we want them to be stacked together in a array. We will use the `paddlenlp.data.Stack` to do that. 
- for every bacthed element (e.g., text, label), we want them to be separated. We can use the `paddlenlp.data.Tuple` function to do that.

Please note that, all these class methods are callable. You can use use them by calling, for example, `Pad()(YourInput)`. But you can also initialize certain values when calling, for example `Pad(axis=0, pad_val=0)(YourInput)`. Moreover, for those RNN models, we will need to create anthoer `Stack()` for the text_seq_len.

In [21]:
from paddlenlp.data import Pad, Stack, Tuple

# The Tuple() will pass the "samples" to the three funcs inside one by one
# then a list of three outputs returned by the three funcs (in this case)
batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=0),  # for text_ids; axis=0, pad_val=0 are default just so you know
        Stack(dtype="int64")  # for label
    ): fn(samples)

In [22]:
# check. Note the following has not been batched.

t, l = batchify_fn(list(map(trans_fn, train_set)))
print("Shape of text_ids preprocessed:", t.shape)
print("Shape of labels preprocessed:", l.shape)

Shape of text_ids preprocessed: (4000, 928)
Shape of labels preprocessed: (4000,)


<a name="5-4"></a>
### Now the dataloader

The dataloader will "load" the numericalized dataset according to our instructions and return an iterable object for the model to loop through during training. As mentioned, we will teach the dataloader how to sample (build batches for) the dataset, and how to further preprocess the built batches. The dataloader will also transform the numericalized dataset into `paddle.Tensor`, making model training quicker (for larger datasets trained on GPUs).

Again, the output cannot be retrieved by index.

In [23]:
from paddle.io import DataLoader

dataloader = DataLoader(
    train,
    batch_sampler=bacth_sampler, 
    collate_fn=batchify_fn)

In [24]:
# check

for d in dataloader:
    print(d)
    break

[Tensor(shape=[64, 191], dtype=int64, place=CPUPlace, stop_gradient=True,
       [[43  , 171 , 468 , ..., 0   , 0   , 0   ],
        [854 , 3   , 569 , ..., 0   , 0   , 0   ],
        [269 , 1568, 2   , ..., 0   , 0   , 0   ],
        ...,
        [40  , 8   , 526 , ..., 0   , 0   , 0   ],
        [5349, 3   , 1174, ..., 0   , 0   , 0   ],
        [310 , 10  , 17  , ..., 0   , 0   , 0   ]]), Tensor(shape=[64], dtype=int64, place=CPUPlace, stop_gradient=True,
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1])]


<a name="6"></a>
## A quick test

It works!

In [25]:
from paddle_models.BoW import BoW

In [26]:
def get_model(model):
    model = paddle.Model(model)
    optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)
    criterion = paddle.nn.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [27]:
model = BoW(len(V.token_to_idx), 2)
model = get_model(model)
%time model.fit(dataloader, epochs=1, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/1


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  return (isinstance(seq, collections.Sequence) and


step 63/63 [==============================] - loss: 0.3530 - acc: 0.7903 - 43ms/step          
CPU times: user 2.93 s, sys: 52.6 ms, total: 2.99 s
Wall time: 2.73 s


<a name="7"></a>
## Wrapped up functions

Before heading to the next section, you can test the following functions/class methods up and see if you can utilize them to do a quick start yourself!

<a name="7-1"></a>
### TextVectorizer

The following wrapped up class method remsembles the one that we built in the `2.1 - wrapped_up_data_preprocessor.ipynb`, but here we are using as many functions as from `paddle`. If you are interested, you can look a look back at the `TextVectorizer` inside the `utils.py` and see if you can create some additional functions (such as save the results into json file for later re-loading). 

In [28]:
from paddlenlp.data import Vocab
import jieba
from collections.abc import Iterable


class TextVectorizer:
     
    def __init__(self, tokenizer=None):
        self.tokenize = tokenizer if tokenizer else jieba.lcut
        self.vocab_to_idx = None
        self.idx_to_vocab = None
        self._V = None
    
    def build_vocab(self, text):
        tokens = list(map(tokenize, text))
        self._V = Vocab.build_vocab(tokens, unk_token='[UNK]', pad_token='[PAD]')
        self.vocab_to_idx = self._V.token_to_idx
        self.idx_to_vocab = self._V.idx_to_token
        
        print('Two vocabulary dictionaries have been built!\n' \
             + 'Please call \033[1mX.vocab_to_idx | X.idx_to_vocab\033[0m to find out more' \
             + ' where [X] stands for the name you used for this TextVectorizer class.')
        
    def text_encoder(self, text):
        if isinstance(text, list):
            return [self(t) for t in text]
        
        tks = self.tokenize(text)
        out = [self.vocab_to_idx[tk] for tk in tks]
        return out
            
    def text_decoder(self, text_ids, sep=" "):
        if all(isinstance(ids, Iterable) for ids in text_ids):
            return [self.text_decoder(ids, sep) for ids in text_ids]
            
        out = []
        for text_id in text_ids:
            out.append(self.idx_to_vocab[text_id])
            
        return f'{sep}'.join(out)
    
    def __call__(self, text):
        if self.vocab_to_idx:
            return self.text_encoder(text)
        raise ValueError("No vocab is built!")

<a name="7-2"></a>
### Example converter

Nothing to change here

In [29]:
def example_converter(example, text_encoder, include_seq_len):
    
    text, label = example
    encoded = text_encoder(text)
    if include_seq_len:
        text_len = len(encoded)
        return encoded, text_len, label
    return encoded, label

<a name="7-3"></a>
### Get trans_fn

Let's customize a method to return trans_fn for us for this series of tutorial!

In [30]:
def get_trans_fn(text_encoder, include_seq_len):
    return lambda ex: example_converter(ex, text_encoder, include_seq_len)

<a name="7-4"></a>
### Get batchify_fn

Let's customize a method to return batchify_fn for us for this series of tutorial!

In [31]:
def get_batchify_fn(include_seq_len):
    
    if include_seq_len:
        stack = [Stack(dtype="int64")] * 2
    else:
        stack = [Stack(dtype="int64")]
    
    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=0),  
        *stack
    ): fn(samples)
    
    return batchify_fn

<a name="7-5"></a>
### Create dataloader 

In [32]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader


def create_dataloader(dataset, 
                      trans_fn, 
                      batchify_fn, 
                      batch_size=64, 
                      shuffle=True, 
                      sampler=BatchSampler):
    
    
    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)
        
    dataset.map(trans_fn)
    batch_sampler = sampler(dataset, 
                            shuffle=shuffle, 
                            batch_size=batch_size)
    
    dataloder = DataLoader(dataset, 
                           batch_sampler=batch_sampler, 
                           collate_fn=batchify_fn)
    
    return dataloder

<a name="8"></a>
## More thorough tests 

This time, we will include the dev_set for validation and the test_set for evaluation!

<a name="8-1"></a>
### Initializations 

In [33]:
from utils import load_dataset, gather_text

train_set, dev_set, test_set = load_dataset(['train.tsv', 'dev.tsv', 'test.tsv'])

text = gather_text(train_set)
V = TextVectorizer()
V.build_vocab(text)

Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


<a name="8-2"></a>
### Test One: CNN

In [34]:
trans_fn = get_trans_fn(V, False)
batchify_fn = get_batchify_fn(False)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn)
dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn)

In [35]:
from paddle_models.CNN import CNN

model = CNN(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_loader, dev_loader, epochs=5, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 63/63 [==============================] - loss: 0.5416 - acc: 0.6322 - 319ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.5524 - acc: 0.7458 - 115ms/step          
Eval samples: 1200
Epoch 2/5
step 63/63 [==============================] - loss: 0.3161 - acc: 0.8458 - 336ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.3629 - acc: 0.8583 - 110ms/step          
Eval samples: 1200
Epoch 3/5
step 63/63 [==============================] - loss: 0.1541 - acc: 0.9457 - 326ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.2716 - acc: 0.8708 - 121ms/step          
Eval samples: 1200
Epoch 4/5
step 63/63 [==============================] - loss: 0.0571 - acc: 0.9885 - 315ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.5115 - acc:

In [36]:
model.evaluate(test_loader)

Eval begin...
step 10/19 - loss: 0.3184 - acc: 0.8984 - 165ms/step
step 19/19 - loss: 0.0751 - acc: 0.8983 - 132ms/step
Eval samples: 1200


{'loss': [0.07513601], 'acc': 0.8983333333333333}

<a name="8-3"></a>
### Test Two: RNN

In [37]:
trans_fn = get_trans_fn(V, True)
batchify_fn = get_batchify_fn(True)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn)
dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn)
test_loader = create_dataloader(test_set, trans_fn, batchify_fn)

In [38]:
from paddle_models.S_RNN import SimpleRNN

model = SimpleRNN(len(V.vocab_to_idx), 2)
model = get_model(model)
%time model.fit(train_loader, dev_loader, epochs=5, verbose=1)

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/5
step 63/63 [==============================] - loss: 0.6814 - acc: 0.5228 - 146ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.6731 - acc: 0.6125 - 57ms/step          
Eval samples: 1200
Epoch 2/5
step 63/63 [==============================] - loss: 0.5233 - acc: 0.7037 - 128ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.5350 - acc: 0.6475 - 48ms/step          
Eval samples: 1200
Epoch 3/5
step 63/63 [==============================] - loss: 0.3629 - acc: 0.9137 - 109ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.6019 - acc: 0.6583 - 47ms/step          
Eval samples: 1200
Epoch 4/5
step 63/63 [==============================] - loss: 0.0705 - acc: 0.9735 - 115ms/step          
Eval begin...
step 19/19 [==============================] - loss: 0.9329 - acc: 0.

In [39]:
model.evaluate(test_loader)

Eval begin...
step 10/19 - loss: 1.3250 - acc: 0.6391 - 74ms/step
step 19/19 - loss: 0.9628 - acc: 0.6675 - 54ms/step
Eval samples: 1200


{'loss': [0.96284074], 'acc': 0.6675}